In [71]:
item_nm = '양파'

In [72]:
import pandas as pd
import glob
import os

# 기본 파일 경로
base_path = f"{item_nm}/유통공사_도매시장_{item_nm}.csv"

# retry 파일 목록 수집
retry_paths = glob.glob(f"{item_nm}/유통공사_retry_{item_nm}_*.csv")

# 기본 파일 읽기
df_list = [pd.read_csv(base_path, encoding="cp949", low_memory=False)]

# retry 파일들 읽기 (있을 경우에만)
for path in retry_paths:
    df_list.append(pd.read_csv(path, encoding="cp949", low_memory=False))

# 병합
df = pd.concat(df_list, ignore_index=True)

# 중복 제거 (전체 행 기준 또는 주요 열 기준)
df = df.drop_duplicates()

# plor_cd가 문자열이 아닐 가능성 대비
df['plor_cd'] = df['plor_cd'].fillna('').astype(str)

pattern = r'^[^0-9]+$'

condition = (
    df['totprc'].isna() | (df['totprc'] <= 0) |
    df['unit_tot_qty'].isna() | (df['unit_tot_qty'] <= 0) |
    df['plor_cd'].str.strip().isin(['0', '0.0']) |
    df['plor_cd'].str.match(pattern, na=False) |
    df['plor_nm'].isna() |
    (df['plor_nm'] == 0)
)

# 조건에 해당하는 행 추출
df_filtered = df[~condition]

# 결과 저장
df_filtered.to_csv(f"{item_nm}/유통공사_도매시장_{item_nm}_결측치제거.csv", index=False, encoding="cp949")

In [73]:
import pandas as pd

# 1. 파일 경로 설정
input_path = f"{item_nm}/유통공사_도매시장_{item_nm}_결측치제거.csv"
output_path = f"{item_nm}/유통공사_도매시장_{item_nm}_한글컬럼명.csv"

# 2. CSV 전체를 문자열로 불러오기 (최초 경고 방지)
df = pd.read_csv(input_path, encoding='cp949', dtype=str)

# 3. 한글 컬럼명 설정
new_columns = [
    '평균가격(원)', '법인코드', '법인이름', '상품 대분류 코드', '상품 대분류 이름',
    '상품 중분류 코드', '상품 중분류 이름', '상품 소분류 코드', '상품 소분류 이름',
    '등급코드', '등급이름', '최고가(원)', '최저가(원)', '포장코드', '포장이름',
    '산지코드', '산지이름', '크기코드', '크기이름', '총가격(원)', '연월일',
    '매매구분', '단위코드', '단위(kg)', '단위물량(kg)', '단위총물량(kg)', '도매시장코드', '도매시장이름'
]

if len(df.columns) != len(new_columns):
    raise ValueError("컬럼 수가 일치하지 않습니다.")

df.columns = new_columns

# 4. 날짜 컬럼 변환
df['연월일'] = pd.to_datetime(df['연월일'], errors='coerce')

# 5. 숫자 컬럼 변환
numeric_columns = ['평균가격(원)', '최고가(원)', '최저가(원)', '총가격(원)', '단위물량(kg)', '단위총물량(kg)']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 6. 혼합 타입 경고 컬럼 처리: NaN을 빈 문자열로, 모두 문자열화
for col in ['포장이름', '산지이름', '크기이름']:
    df[col] = df[col].fillna('').astype(str)

# 7. CSV 저장 (완전 정리된 상태)
df.to_csv(output_path, index=False, encoding='cp949')

In [74]:
import pandas as pd

# 파일 경로
input_path = f"{item_nm}/유통공사_도매시장_{item_nm}_한글컬럼명.csv"
output_path = f"{item_nm}/유통공사_{item_nm}_요약데이터.csv"

# CSV 파일 불러오기
df = pd.read_csv(input_path, encoding='cp949')

selected_columns = ['연월일', '상품 중분류 코드', '상품 중분류 이름', '상품 소분류 코드', '상품 소분류 이름',
    '등급코드', '등급이름', '총가격(원)', '단위총물량(kg)', '산지코드', '산지이름']
df_selected = df[selected_columns]
df_selected.rename(columns={
    '상품 중분류 코드':'품목코드',
    '상품 중분류 이름':'품목명',
    '상품 소분류 코드':'품종코드',
    '상품 소분류 이름':'품종명',
    '총가격(원)':'총금액(원)',
    '단위총물량(kg)':'총거래량(kg)'
}, inplace=True)

df_selected['평균단가(원)'] = df_selected['총금액(원)'] / df_selected['총거래량(kg)']
df_selected['평균단가(원)'] = df_selected['평균단가(원)'].replace([float('inf'), -float('inf')], None)
df_selected.insert(df_selected.columns.get_loc('총거래량(kg)') + 1, '평균단가(원)', df_selected.pop('평균단가(원)'))

# 새 CSV로 저장
df_selected.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1634698712.py:8: DtypeWarning: Columns (9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path, encoding='cp949')
C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1634698712.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.rename(columns={
C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1634698712.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['평균단가(원)'] = df_selected['총금액(원)'] / df_selected['총거래량(kg)']
C:\Users\Admin\AppDa

In [75]:
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding = 'cp949')

df.isna().sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\298332032.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding = 'cp949')


연월일         0
품목코드        0
품목명         0
품종코드        0
품종명         0
등급코드        0
등급이름        0
총금액(원)      0
총거래량(kg)    0
평균단가(원)     0
산지코드        0
산지이름        0
dtype: int64

In [76]:
import pandas as pd
import re

# CSV 파일 로드
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding='cp949')

# 상품 중분류 이름 결측치 처리
df['품목코드'] = df['품목코드'].replace('', pd.NA).fillna(item_nm)
df['품목명'] = df['품목명'].replace('', pd.NA).fillna(item_nm)
df['품종코드'] = df['품종코드'].replace('', pd.NA).fillna(item_nm)
df['품종명'] = df['품종명'].replace('', pd.NA).fillna(item_nm)

# 산지코드 정제 함수 (문자 → 0, 앞자리 0 제거, 6자리 맞춤)
def clean_origin_code_backpad(code):
    code_str = str(code).strip()
    replaced = re.sub(r'\D', '0', code_str)      # 문자 → 0
    stripped = replaced.lstrip('0')              # 앞쪽 0 제거
    if not stripped:  # 모두 0이거나 제거 결과 없음
        stripped = '0'
    return stripped[:6].ljust(6, '0')            # 앞 6자리 + 뒤 0 패딩

# 적용
df['산지코드'] = df['산지코드'].apply(clean_origin_code_backpad)

# 저장
output_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"
df.to_csv(output_path, index=False, encoding='cp949')

print(f"정제된 데이터가 저장되었습니다: {output_path}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\57156428.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터.csv", encoding='cp949')


정제된 데이터가 저장되었습니다: 양파/유통공사_양파_요약데이터_정제.csv


In [77]:
df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv", encoding = 'cp949')

df.isna().sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1449754255.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv", encoding = 'cp949')


연월일         0
품목코드        0
품목명         0
품종코드        0
품종명         0
등급코드        0
등급이름        0
총금액(원)      0
총거래량(kg)    0
평균단가(원)     0
산지코드        0
산지이름        0
dtype: int64

In [78]:
import pandas as pd

# 파일 경로 설정
file_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"

# CSV 파일 읽기 (경고 방지를 위해 low_memory=False 사용)
df = pd.read_csv(file_path, encoding="cp949", low_memory=False)

# 산지코드가 문자열로 처리되도록 변환
df['산지코드'] = df['산지코드'].fillna('').astype(str)

# 필터링 조건: totprc 또는 unit_tot_qty가 NaN이거나 0인 경우
pattern = r'^[^0-9]+$'

condition = (
    (df['산지코드'] == '') |
    (df['산지코드'].str.strip() == "0") |
    (df['산지코드'].str.strip() == "0.0")
#     df['plor_cd'].str.match(pattern, na=False)) 
#     (df['plor_nm'].isna() | (df['plor_nm'] == 0))
)

# 조건에 해당하는 행 추출
df_filtered = df[condition]

# 결과 저장
df_filtered.to_csv(f"{item_nm}/결측치_{item_nm}.csv", index=False, encoding="cp949")

In [79]:
import pandas as pd

# 1. 파일 경로 설정
summary_path = f"{item_nm}/유통공사_{item_nm}_요약데이터_정제.csv"
region_code_path = "표준코드/산지코드_직팜.csv"

# 2. CSV 불러오기
summary_df = pd.read_csv(summary_path, encoding='cp949')
region_df = pd.read_csv(region_code_path, encoding='cp949')

# 3. 산지코드 범위 파싱 함수
def parse_code_range(code_str):
    if "~" in code_str:
        start, end = code_str.split("~")
    else:
        start = end = code_str
    return int(start), int(end)

# 4. 산지코드 범위 숫자 컬럼 생성
region_df[['start_code', 'end_code']] = region_df['산지코드'].apply(
    lambda x: pd.Series(parse_code_range(x))
)

# 5. 산지코드 → 직팜산지코드/이름 매핑 테이블 확장
expanded_rows = []
for _, row in region_df.iterrows():
    for code in range(row['start_code'], row['end_code'] + 1):
        expanded_rows.append({
            '산지코드': code,
            '직팜산지코드': row['직팜산지코드'],
            '직팜산지이름': row['직팜산지이름']
        })
expanded_map_df = pd.DataFrame(expanded_rows)

# 6. 요약 데이터 산지코드 정수형으로 변환
summary_df['산지코드'] = pd.to_numeric(summary_df['산지코드'], errors='coerce').astype('Int64')
summary_df = summary_df[summary_df['산지코드'].notna()].copy()
summary_df['산지코드'] = summary_df['산지코드'].astype(int)

# 7. 직팜 컬럼 제거 후 병합
summary_df = summary_df.drop(columns=['직팜산지코드', '직팜산지이름'], errors='ignore')
summary_df = pd.merge(summary_df, expanded_map_df, on='산지코드', how='left')

# 8. 산지이름이 누락된 경우 직팜산지이름으로 보완
summary_df['산지이름'] = summary_df['산지이름'].fillna(summary_df['직팜산지이름'])


# 9. 결과 저장
summary_df.to_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\3747483502.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  summary_df = pd.read_csv(summary_path, encoding='cp949')


In [80]:
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding = 'cp949')

df.isna().sum()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\70669388.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding = 'cp949')


연월일         0
품목코드        0
품목명         0
품종코드        0
품종명         0
등급코드        0
등급이름        0
총금액(원)      0
총거래량(kg)    0
평균단가(원)     0
산지코드        0
산지이름        0
직팜산지코드      0
직팜산지이름      0
dtype: int64

In [81]:
import pandas as pd

# CSV 파일 로드
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding='cp949')

# 등급이름이 NaN 또는 공백이거나 '.'인 경우 '보통'으로 대체
df['등급이름'] = df['등급이름'].replace('.', '보통').fillna('보통')
df['등급이름'] = df['등급이름'].replace(r'^\s*$', '보통', regex=True)

# '특', '상', '보통' 외의 값은 모두 '하'로 변경
df['등급이름'] = df['등급이름'].apply(lambda x: x if x in ['특', '상', '보통'] else '하')

# 저장
output_path = f"{item_nm}/{item_nm}요약데이터_직팜정리.csv"
df.to_csv(output_path, index=False, encoding='cp949')

print(f"등급이름 수정 완료: {output_path}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1157110577.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜산지정리.csv", encoding='cp949')


등급이름 수정 완료: 양파/양파요약데이터_직팜정리.csv


In [82]:
import pandas as pd

# 파일 경로와 인코딩으로 데이터 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜정리.csv", encoding='cp949')

# 결측치가 하나라도 있는 행만 추출
df_na = df[df.isna().any(axis=1)]

# 결과 저장
output_path = f"{item_nm}/{item_nm}_결측치포함행.csv"
df_na.to_csv(output_path, index=False, encoding='cp949')

print(f"결측치 포함 행 {len(df_na)}개를 '{output_path}'로 저장했습니다.")

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\2818479368.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜정리.csv", encoding='cp949')


결측치 포함 행 0개를 '양파/양파_결측치포함행.csv'로 저장했습니다.


In [83]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜정리.csv", encoding='cp949')

# 제거할 컬럼 리스트
columns_to_drop = ['산지코드', '산지이름', '직팜산지이름']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 새로 추가할 컬럼들 (초기값은 None)
new_columns = [
    '휴일여부', '명절지수', '작기정보'
]
for col in new_columns:
    df[col] = None

# 결과 저장
df.to_csv(f"{item_nm}/{item_nm}요약데이터_컬럼정리완료.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\649520230.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_직팜정리.csv", encoding='cp949')


In [84]:
import pandas as pd

# 파일 경로
summary_path = f"{item_nm}/{item_nm}요약데이터_컬럼정리완료.csv"
mapping_path = "기상/산지코드_직팜_관측지점_매핑완료.csv"

# CSV 파일 불러오기
df_summary = pd.read_csv(summary_path, encoding='cp949')
df_mapping = pd.read_csv(mapping_path, encoding='cp949')

# 중복 제거
df_mapping = df_mapping.drop_duplicates(subset='직팜산지코드')

# '직팜산지코드' 기준으로 '관측지점' 컬럼 병합
df_merge = pd.merge(
    df_summary,
    df_mapping[['직팜산지코드', '관측지점']],
    on='직팜산지코드',
    how='left'
)

# '작기정보' 컬럼 오른쪽에 '관측지점' 컬럼 삽입
if '작기정보' in df_merge.columns:
    insert_idx = df_merge.columns.get_loc('작기정보') + 1  # '작기정보' 다음 위치
    관측지점_series = df_merge.pop('관측지점')
    df_merge.insert(insert_idx, '관측지점', 관측지점_series)

# 결과 저장
df_merge.to_csv(f"{item_nm}/{item_nm}요약데이터_관측지점추가.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1469962328.py:8: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_summary = pd.read_csv(summary_path, encoding='cp949')


In [85]:
import pandas as pd

# 파일 경로 설정
crop_path = f"{item_nm}/{item_nm}요약데이터_관측지점추가.csv"
weather_path = "기상/기상청_일기요소.csv"
output_path = f"{item_nm}/{item_nm}요약데이터_기상병합완료.csv"

# 데이터 불러오기
df_crop = pd.read_csv(crop_path, encoding='cp949')
df_weather = pd.read_csv(weather_path, encoding='cp949')

# 날짜 처리 → YYYYMMDD 문자열
df_crop['연월일'] = pd.to_datetime(df_crop['연월일'], errors='coerce')
df_crop['날짜'] = df_crop['연월일'].dt.strftime('%Y%m%d')

df_weather['TM'] = pd.to_datetime(df_weather['TM'], errors='coerce')
df_weather['날짜'] = df_weather['TM'].dt.strftime('%Y%m%d')

# 관측지점/STN 모두 int로 통일 후 str 변환
df_crop['관측지점'] = df_crop['관측지점'].astype('Int64').astype(str)
df_weather['STN'] = df_weather['STN'].astype('Int64').astype(str)

# 병합 키 생성
df_crop['merge_key'] = df_crop['날짜'] + '_' + df_crop['관측지점']
df_weather['merge_key'] = df_weather['날짜'] + '_' + df_weather['STN']

# 날씨 컬럼 매핑
weather_cols = {
    'TA_AVG': '일평균기온',
    'TA_MAX': '최고기온',
    'TA_MIN': '최저기온',
    'HM_AVG': '평균상대습도',
    'RN_DAY': '강수량(mm)',
    'RN_60M_MAX': '1시간최고강수량(mm)'
}

# 병합용 데이터 준비
df_weather_subset = df_weather[['merge_key'] + list(weather_cols.keys())].copy()
df_weather_subset.rename(columns=weather_cols, inplace=True)

# 병합 수행
df_merged = pd.merge(df_crop, df_weather_subset, on='merge_key', how='left')

# 정리: 병합키 및 중간 날짜 컬럼 제거
df_merged.drop(columns=['merge_key', '날짜'], inplace=True)

# <NA> → 빈 문자열, NaN → 빈 문자열 처리
df_merged.replace("<NA>", "", inplace=True)
df_merged.fillna("", inplace=True)

# 결과 저장
df_merged.to_csv(output_path, index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1151683674.py:9: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crop = pd.read_csv(crop_path, encoding='cp949')


In [86]:
import pandas as pd

# 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상병합완료.csv", encoding='cp949')

# 제거할 컬럼 리스트
columns_to_drop = ['관측지점']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 결과 저장
df.to_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\1034814857.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상병합완료.csv", encoding='cp949')


In [87]:
import pandas as pd

df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", encoding='cp949')

df = df[~(df['직팜산지코드'] == 1161)]

df.to_csv(f"{item_nm}/{item_nm}요약데이터_기상_수입산제거.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\2231650891.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상.csv", encoding='cp949')


In [88]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상_수입산제거.csv", encoding='cp949')

# 연월일을 datetime으로 변환
df['연월일'] = pd.to_datetime(df['연월일'], errors='coerce')

# 평균단가 컬럼 이름 자동 감지 (혹시 이름이 다를 경우 대비)
column_name = '평균단가'
if column_name not in df.columns:
    column_name = df.columns[df.columns.str.contains('단가|가격')][0]

# 날짜별 이상치 검출
outlier_rows = []
for date, group in df.groupby('연월일'):
    if len(group) < 4:  # 너무 적은 그룹은 건너뜀
        continue
        
    q1 = group[column_name].quantile(0.25)
    q3 = group[column_name].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outliers = group[(group[column_name] < lower) | (group[column_name] > upper)]
    outlier_rows.append(outliers)

# 이상치 모음
datewise_outliers = pd.concat(outlier_rows, ignore_index=True)

# 이상치 제거된 데이터셋 생성
df_cleaned = df.drop(index=datewise_outliers.index)

# (선택) CSV로 저장
df_cleaned.to_csv(f"{item_nm}/{item_nm}_이상치제거완료.csv", index=False, encoding='cp949')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6600\3086585174.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{item_nm}/{item_nm}요약데이터_기상_수입산제거.csv", encoding='cp949')
